In [2]:
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_experimental.tools.python.tool import PythonREPLTool
import os
import yfinance as yf
from forex_python.converter import CurrencyRates


In [3]:

from dotenv import load_dotenv

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"], model_name="llama3-70b-8192")


In [4]:

def currency_converter(query: str) -> str:
    try:
        c = CurrencyRates()
        parts = query.lower().split()
        amount = float(parts[0])
        from_currency = parts[1].upper()
        to_currency = parts[-1].upper()
        result = c.convert(from_currency, to_currency, amount)
        return f"{amount} {from_currency} = {result:.2f} {to_currency}"
    except Exception as e:
        return f"[Currency Converter] Error: {e}"

def stock_analyzer(ticker: str) -> str:
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="5d")
        if hist.empty:
            return "[Stock Analyzer] No data found."
        last_price = hist['Close'][-1]
        return f"The latest closing price of {ticker.upper()} is ${last_price:.2f}"
    except Exception as e:
        return f"[Stock Analyzer] Error: {e}"



In [5]:

tools = [
    Tool(name="Python REPL", func=PythonREPLTool().run, description="Useful for doing math or logic in Python"),
    Tool(name="Currency Converter", func=currency_converter, description="Converts currency like '100 USD to INR'"),
    Tool(name="Stock Analyzer", func=stock_analyzer, description="Analyzes stock price. Example input: 'AAPL' or 'TSLA'")
]


In [6]:

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


C:\Users\moham\AppData\Local\Temp\ipykernel_4932\3776416932.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [7]:

query = "what is the radius of the Earth surface"
result = agent.run(query)


C:\Users\moham\AppData\Local\Temp\ipykernel_4932\1715144060.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run(query)




> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I need to find the radius of the Earth's surface.

Action: Python REPL
Action Input: `import math; print(math.pi * 6371)` (assuming the Earth is a perfect sphere with a circumference of 40,075 km)
Observation: SyntaxError('leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers', ('<string>', 1, 104, 'import math; print(math.pi * 6371)` (assuming the Earth is a perfect sphere with a circumference of 40,075 km)', 1, 105))
Thought:It looks like I made a mistake in my Python code.

Action: Python REPL
Action Input: `import math; print(math.pi * 6371)`
Observation: 20015.086796020572

Thought:I now know the final answer

Final Answer: The radius of the Earth's surface is approximately 20015.09 kilometers, or 6371 kilometers (since circumference = 2 * pi * radius).

> Finished chain.


In [8]:

print("\n🧠 Final Answer:\n", result)


🧠 Final Answer:
 The radius of the Earth's surface is approximately 20015.09 kilometers, or 6371 kilometers (since circumference = 2 * pi * radius).
